In [1]:
import sys
import os
from os.path import join as pj
# from os.path import exists

_HOME_DIR = os.path.expanduser("~")    

try:
    _SPARCFIRE_DIR = os.environ["SPARCFIRE_HOME"]
    _MODULE_DIR = pj(_SPARCFIRE_DIR, "GalaxyMusic", "GalfitModule")
except KeyError:
    # print("SPARCFIRE_HOME is not set. Please run 'setup.bash' inside SpArcFiRe directory if not done so already.")
    # print("Running on the assumption that GalfitModule is in your home directory... (if not this will fail and quit!)") 
    _MODULE_DIR = pj(_HOME_DIR, "GalaxyMusic", "GalfitModule")
    
sys.path.append(_MODULE_DIR)
    
from Classes.Components import *
from Classes.Containers import *
from Functions.helper_functions import *
from sparc_to_galfit_feedme_gen import *

In [2]:
from astropy.io import fits
from astropy.table import Table
from shutil import copy2

In [59]:
# outside loop
g_prefix = "galaxy"
g_number = 0

IN_DIR  = pj(_HOME_DIR, "GalaxyMusic", "music-in")
TMP_DIR = pj(_HOME_DIR, "GalaxyMusic", "music-tmp")
OUT_DIR = pj(_HOME_DIR, "GalaxyMusic", "music-out")

run_galfit = pj(_MODULE_DIR, "galfit")


# in loop
gname = f"{g_prefix}_{g_number}"
filename = f"{pj(IN_DIR, gname)}.fits"
template = pj(_MODULE_DIR, "template.fits")
sp(f"cp {template} {filename}")
#gen_template(filename)

if not exists(pj(OUT_DIR, gname)):
    os.mkdir(pj(OUT_DIR, gname))
    
feedme = pj(OUT_DIR, gname, f"{gname}.in")
galaxy_feedme = FeedmeContainer(path_to_feedme = feedme)

galaxy_feedme.header.param_values.update({
    "input_image"  : filename,
    "output_image" : pj(OUT_DIR, gname, f"{gname}.fits"),
    "optimize"     : 1,
    "region_to_fit" : (0,148,0,148)
    }
)

galaxy_feedme.bulge.param_values.update(new)
galaxy_feedme.disk.param_values.update(new)

galaxy_feedme.update_components(bulge = bulge, disk = disk)

#print(galaxy_feedme)
_ = sp(f"{run_galfit} {feedme}")